In [1]:
from registry.client import ModelRegistryClient
import os
import io
from datetime import datetime
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from registry.schemas import ModelMetadata
import numpy as np
import pickle

In [ ]:
client = ModelRegistryClient(base_url="http://localhost:8000")
client.health_check()


In [ ]:
np.random.seed(42)
X = np.random.rand(100, 2)
y = np.random.randint(0, 2, 100)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LogisticRegression()
model.fit(X_train, y_train)


model_buffer = io.BytesIO()
pickle.dump(model, model_buffer)
model_buffer.seek(0)


In [ ]:
metadata = ModelMetadata(
            id="model123",
            name="example_model",
            version="1.0.0",
            file_extension="pkl",
            storage_group="ml-models",
            description="Example model",
            framework="scikit-learn",
            metrics={"accuracy": 0.95},
            parameters={"n_estimators": 100}
        )
res = client.upload_model(model_buffer, metadata)
print(res)


In [ ]:

file_path = res.file_path
metadata_id = res.metadata_id
res = client.get_model_buffer(file_path, metadata_id)
model = pickle.load(res[0])

In [ ]:
print(res[1])


### Model Registry Module

In [29]:
from registry.registry import ModelRegistry
from registry.schemas import ModelMetadata
import pickle
import numpy as np

# Create sample model
X = np.random.rand(100, 2)
y = np.random.randint(0, 2, 100)
sample_model = {"data": (X, y)}

with open("test_model.pkl", "wb") as f:
    pickle.dump(sample_model, f)


In [ ]:
model_bufferr = io.BytesIO()
pickle.dump(sample_model,model_bufferr)
model_bufferr.seek(0)

In [ ]:
metadata = ModelMetadata(
    id= datetime.now().strftime("%d%m%Y%H%M%S"),
    name="memory_test_model",
    file_extension=".pkl",
    storage_group="test-project",
    version="1.0.0",
    framework="sk",
    description="In-memory test of logistic regression model",
    metrics={
        "train_accuracy": float(0.6),
        "test_accuracy": float(0.6)
    },
    parameters={
        "n_features": X.shape[1],
        "n_samples": X.shape[0],
        "model_type": "LogisticRegression",
        "model_params": {"a":12}
    }
)


model_reg = ModelRegistry()

res = model_reg.register_model(model_bufferr,metadata.model_dump())
res


### API Testing

In [34]:
from datetime import datetime
import io
import json
import pickle
import requests

import numpy as np
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from registry.schemas import ModelMetadata

In [ ]:
mode = 1 # 1 is pickle

if mode == 1:

    # Create sample model
    X = np.random.rand(100, 2)
    y = np.random.randint(0, 2, 100)
    sample_model = {"data": (X, y)}

    with open("test_model.pkl", "wb") as f:
        pickle.dump(sample_model, f)

    # First, let's fix the ModelMetadata instance
    metadata = ModelMetadata(
        id=datetime.now().strftime("%d%m%Y%H%M%S"),
        name="memory_test_model",
        file_extension="pkl",  # Remove the dot (.) from extension
        storage_group="test-project",
        version="1.0.0",
        framework="sk",
        description="In-memory test of logistic regression model",
        metrics={
            "train_accuracy": float(0.6),
            "test_accuracy": float(0.6)
        },
        parameters={
            "n_features": X.shape[1],
            "n_samples": X.shape[0],
            "model_type": "LogisticRegression",
            "model_params": {"a": 12}
        }
    )



    # Create in-memory model buffer
    model_buffer = io.BytesIO()
    pickle.dump(sample_model, model_buffer)
    model_buffer.seek(0)

    # Prepare files for upload
    files = {
        'metadata': (None, json.dumps(metadata.model_dump())),  # Convert to dict first, then to JSON
        'model_file': (
            'model.pkl',
            model_buffer,
            'application/octet-stream'
        )
    }

    # Make request
    response = requests.post(
        url='http://localhost:8000/models/upload',
        files=files,
        headers={'accept': 'application/json'}
    )


else:
# Create and train a proper ML model
    # Generate sample data
    np.random.seed(42)
    X = np.random.rand(100, 2)
    y = np.random.randint(0, 2, 100)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Calculate metrics
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)

    # Convert to ONNX
    initial_type = [('float_input', FloatTensorType([None, 2]))]
    onnx_model = convert_sklearn(model, initial_types=initial_type)

    # Create metadata
    metadata = ModelMetadata(
        id=datetime.now().strftime("%d%m%Y%H%M%S"),
        name="onnx_test_model",
        file_extension="onnx",
        storage_group="test-project",
        version="1.0.0",
        framework="onnx",
        description="ONNX converted logistic regression model",
        metrics={
            "train_accuracy": float(train_accuracy),
            "test_accuracy": float(test_accuracy)
        },
        parameters={
            "n_features": X.shape[1],
            "n_samples": X.shape[0],
            "model_type": "LogisticRegression",
            "model_params": model.get_params()
        }
    )

    # Create in-memory model buffer
    model_buffer = io.BytesIO()
    model_buffer.write(onnx_model.SerializeToString())
    model_buffer.seek(0)

    # Prepare files for upload
    files = {
        'metadata': (None, json.dumps(metadata.model_dump())),
        'model_file': (
            'model.onnx',
            model_buffer,
            'application/octet-stream'
        )
    }

    # Make request
    response = requests.post(
        url='http://localhost:8000/models/upload',
        files=files,
        headers={'accept': 'application/json'}
    )


print("API Response :",response)
print(f"Upload response status code: {response.status_code}")
print(f"Response content: {response.json()}")


In [58]:
# Make request
response = requests.post(
    url='http://localhost:8000/models/upload',
    files=files,
    headers={'accept': 'application/json'}
)

In [ ]:
response_json = response.json()
response_json

In [66]:
file_path =  response_json["storage_path"]
metadata_id = response_json["metadata_id"]
url = "http://localhost:8000"

In [67]:
response = requests.get(f"http://localhost:8000/models/metadata/{file_path}?metadata_id={metadata_id}")

In [70]:
response = requests.get(f"http://localhost:8000/models/file/{file_path}?metadata_id={metadata_id}")

In [ ]:
for i in response.iter_content():
    print(i)